# Distance diagnostics

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

import dask.dataframe as dd
import os
from glob import glob

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.geodesic as cgeo

import matplotlib.transforms as mtransforms
crs = ccrs.PlateCarree()

import histlib.matchup as match
from  histlib.diagnosis import build_matchup_dataframe, synthetic_figure, path_csv, put_fig_letter, build_diagnostic
from histlib.cstes import labels, zarr_dir, matchup_dir, c0

/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


In [2]:
cluster.close()

NameError: name 'cluster' is not defined

____________
# Create data
## Per labels

In [2]:
paths0 = glob(os.path.join(f'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/matchup/matchup_*.zarr'))
paths1 = glob(os.path.join(f'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/cutoff_matchup/cutoff_matchup_*_2.zarr'))

In [5]:
def path_csv_per_labels(comb, drifter_type, drogue_status, DT, l):
    # Drogue status
    if drogue_status == 'both' : dr_st = 'bothdroguedundrogued'
    elif drogue_status : dr_st = 'drogued'
    elif not drogue_status : dr_st = 'undrogued'

    #Drifter type
    if drifter_type == 'both' : dr = 'bothgpsargos'
    elif drifter_type == 'gps' : dr = 'gps'
    elif drifter_type == 'argos' : dr = 'argos'
    #dt
    if DT : dt=str(int(DT/60))
    if DT ==None : dt='None'

    return os.path.join(zarr_dir,'analysis_files', 'per_labels',f'{l}__{dr_st}__{dr}__{dt}min__'+'__'.join([comb[key] for key in comb])+'.csv')


def build_matchup_dataframe_per_labels(comb, drifter_type = 'both', drogue_status=True, DT=None):
    from histlib.cstes import labels
    # gps/argos
    if drifter_type!='both': 
        labels = [l for l in labels if drifter_type in l]
    for l in labels :
        paths = [f'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/matchup/matchup_{l}.zarr'] + glob(os.path.join(f'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/cutoff_matchup/cutoff_matchup_{l}*.zarr'))
        df = xr.merge([xr.open_dataset(path) for path in paths])[[comb[key] for key in comb]+['drogue_status']].to_dataframe()
        df['drifter_type'] = l.split('_')[0]
        df['drogue_status'] = df['drogue_status'].astype('bool')
        
        if drogue_status != 'both':
            df = df.where(df.drogue_status==drogue_status).dropna()
        if DT :
            df =df.where(df.alti___time_difference<DT).dropna()

        df = build_diagnostic(df, comb)
        df = df.set_index('obs')
        path = path_csv_per_labels(comb, drifter_type, drogue_status, DT, l)
        df.to_csv(path)
        print(l)


In [12]:
DL, DT = 25e3, 30*60
# if not already generated
comb_av_d = {'acc':'drifter_acc_x_25', 'cor':'drifter_coriolis_x_25', 'ggx':'aviso_drifter_ggx_adt', 'wd':'es_cstrio_z15_drifter_wd_x'}
comb_av_al = {'acc':'drifter_acc_x_25', 'cor':'drifter_coriolis_x_25', 'ggx':'aviso_alti_ggx_adt', 'wd':'es_cstrio_z15_drifter_wd_x'}
comb_alg = {'acc':'drifter_acc_x_25','cor':'drifter_coriolis_x_25','ggx':'alti_ggx_adt_filtered_ocean_tide_internal_tide','wd':'es_cstrio_z15_drifter_wd_x'}

comb = comb_alg
build_matchup_dataframe_per_labels(comb, drifter_type = 'both', drogue_status=True, DT=30*60)


gps_Jason-3_2020
argos_Jason-3_2020
gps_SARAL_2020
argos_SARAL_2020
gps_Cryosat-2_2020
argos_Cryosat-2_2020
gps_Sentinel-3_A_2020
argos_Sentinel-3_A_2020
gps_Sentinel-3_B_2020
argos_Sentinel-3_B_2020
gps_Jason-3_2019
argos_Jason-3_2019
gps_SARAL_2019
argos_SARAL_2019
gps_Cryosat-2_2019
argos_Cryosat-2_2019
gps_Sentinel-3_A_2019
argos_Sentinel-3_A_2019
gps_Sentinel-3_B_2019
argos_Sentinel-3_B_2019
gps_Jason-3_2018
argos_Jason-3_2018
gps_SARAL_2018
argos_SARAL_2018
gps_Cryosat-2_2018
argos_Cryosat-2_2018
gps_Sentinel-3_A_2018
argos_Sentinel-3_A_2018
gps_Sentinel-3_B_2018
argos_Sentinel-3_B_2018
gps_Jason-3_2017
argos_Jason-3_2017
gps_Jason-2_2017
argos_Jason-2_2017
gps_SARAL_2017
argos_SARAL_2017
gps_Cryosat-2_2017
argos_Cryosat-2_2017
gps_Sentinel-3_A_2017
argos_Sentinel-3_A_2017
gps_Jason-3_2016
argos_Jason-3_2016
gps_Jason-2_2016
argos_Jason-2_2016
gps_SARAL_2016
argos_SARAL_2016
gps_Cryosat-2_2016
argos_Cryosat-2_2016
gps_Sentinel-3_A_2016
argos_Sentinel-3_A_2016
gps_Jason-2_2015
arg

## One file for all with good types

In [17]:
comb_av_d = {'acc':'drifter_acc_x_25', 'cor':'drifter_coriolis_x_25', 'ggx':'aviso_drifter_ggx_adt', 'wd':'es_cstrio_z15_drifter_wd_x'}
comb_av_al = {'acc':'drifter_acc_x_25', 'cor':'drifter_coriolis_x_25', 'ggx':'aviso_alti_ggx_adt', 'wd':'es_cstrio_z15_drifter_wd_x'}
comb_alg = {'acc':'drifter_acc_x_25','cor':'drifter_coriolis_x_25','ggx':'alti_ggx_adt_filtered_ocean_tide_internal_tide','wd':'es_cstrio_z15_drifter_wd_x'}

comb = comb_alg
paths = glob(os.path.join(zarr_dir,'analysis_files', 'per_labels', '*'+'__'.join([comb[key] for key in comb])+'*'))
vars_errors = ['B_acc','E_acc','E_cor', 'B_cor', 'E_ggx', 'B_ggx', 'E_wd', 'B_wd',
 'X_acc_cor', 'X_acc_ggx', 'X_acc_wd', 'X_cor_ggx', 'X_cor_wd','X_ggx_wd', 
 'ACC', 'COR', 'GGX', 'WD', 'S', 'sigma']
df= dd.read_csv(paths)[vars_errors+['alti___distance', 'obs']].repartition(npartitions=10).persist()
df_ = df.compute()
for v in vars_errors+['alti___distance'] :
    df_[v] = df_[v].astype('float64')
path = path_csv(comb, 'both',  True, 200e3,30*60)
path

'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok/analysis_files/drogued__bothgpsargos__200000km__30min__drifter_acc_x_25__drifter_coriolis_x_25__alti_ggx_adt_filtered_ocean_tide_internal_tide__es_cstrio_z15_drifter_wd_x.csv'

In [18]:
df_.to_csv(path)

__________
___________
# Compute mean

In [24]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    #cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    #cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=5, processes=5, walltime='00:30:00')
    w = cluster.scale(jobs=6)
else:
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.0.28:8787/status,
Dashboard: http://10.148.0.28:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.28:44553,Workers: 0
Dashboard: http://10.148.0.28:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [25]:
 cluster.close()

In [22]:
def mean_df(df):
    return df.mean()
from scipy.stats import bootstrap
def compute_bootstrap_error(dff):
    #print(len(dff))
    if len(dff)<3:
        return np.nan
    else : 
        data = (dff, )  # samples must be in a sequence
        print('ok')
        return bootstrap(data, statistic = mean_df).standard_error

def dist_stats(df, dl=5e3, vars_errors=['S']):
    df["distbin"] = (df.alti___distance // dl) * dl

    #df0 = dd.from_pandas(df, npartitions=10).persist()

    # Mean in bins
    mean = df.groupby(["distbin"]).mean().compute()
    
    #count
    count = df.reset_index()[['B_acc', 'distbin']].groupby(["distbin"]).count().B_acc.compute().rename('nb_coloc_bin')

    # bootstrap errors
    DF = []
    for v in vars_errors :
        DF.append(df.reset_index()[[v, 'distbin']].groupby(["distbin"])[v].apply(compute_bootstrap_error).compute())
    booterrors = pd.concat(DF, axis=1)
    booterrors = booterrors.rename(columns={v:'be__'+v for v in booterrors.columns})

    return pd.concat([mean, count, booterrors], axis=1)

In [21]:
comb_av_d = {'acc':'drifter_acc_x_25', 'cor':'drifter_coriolis_x_25', 'ggx':'aviso_drifter_ggx_adt', 'wd':'es_cstrio_z15_drifter_wd_x'}
comb_av_al = {'acc':'drifter_acc_x_25', 'cor':'drifter_coriolis_x_25', 'ggx':'aviso_alti_ggx_adt', 'wd':'es_cstrio_z15_drifter_wd_x'}
comb_alg = {'acc':'drifter_acc_x_25','cor':'drifter_coriolis_x_25','ggx':'alti_ggx_adt_filtered_ocean_tide_internal_tide','wd':'es_cstrio_z15_drifter_wd_x'}

for comb in [comb_av_d, comb_av_al, comb_alg] : 
    path = path_csv(comb, 'both', True, 200e3, 30*60)
    df = dd.read_csv(path).repartition(npartitions=380).drop(columns=('Unnamed: 0')).persist()
    
    pathm = os.path.join(zarr_dir,'analysis_files', 'dist_mean___'+'__'.join([comb[key] for key in comb])+'.csv')
    vars_errors = ['B_acc', 'E_acc', 'E_cor', 'B_cor', 'E_ggx', 'B_ggx', 'E_wd', 'B_wd',
     'X_acc_cor', 'X_acc_ggx', 'X_acc_wd', 'X_cor_ggx', 'X_cor_wd','X_ggx_wd', 
     'ACC', 'COR', 'GGX', 'WD', 'S', 'sigma']
    dfm = dist_stats(df, vars_errors = vars_errors)
    dfm.to_csv(pathm)

In [13]:
pathm = os.path.join(zarr_dir,'analysis_files', 'dist_mean___'+'__'.join([comb[key] for key in comb])+'.csv')

In [ ]:
dff  = pd.read_csv(pathm)

In [31]:
dff

,distbin,B_acc,E_cor,B_cor,E_ggx,B_ggx,E_wd,B_wd,X_acc_cor,X_acc_ggx,...,be__X_acc_wd,be__X_cor_ggx,be__X_cor_wd,be__X_ggx_wd,be__ACC,be__COR,be__GGX,be__WD,be__S,be__sigma
0,0.0,8.153991e-11,5.501772e-11,3.311115e-10,5.111757e-11,2.660701e-10,2.657518e-12,1.436640e-11,1.210335e-10,4.480942e-11,...,9.845599e-13,1.930655e-11,2.532240e-12,2.077152e-12,2.579460e-12,1.123666e-11,9.591435e-12,6.586723e-13,3.343467e-12,2.078044e-11
1,2000.0,7.940268e-11,4.348054e-11,3.207830e-10,5.790140e-11,2.632196e-10,2.682211e-12,1.369412e-11,1.094436e-10,5.090070e-11,...,6.117999e-13,1.306138e-11,1.551919e-12,1.249264e-12,1.685713e-12,7.060838e-12,7.079343e-12,4.252774e-13,2.175636e-12,1.377199e-11
2,4000.0,8.085761e-11,5.385219e-11,3.202042e-10,6.357749e-11,2.682823e-10,1.768999e-12,1.461558e-11,1.087772e-10,5.559775e-11,...,6.521208e-13,1.371672e-11,1.573659e-12,1.304120e-12,1.572540e-12,7.940660e-12,7.752538e-12,4.894581e-13,4.346569e-12,1.492473e-11
3,6000.0,8.344258e-11,5.343818e-11,3.151295e-10,5.908416e-11,2.554294e-10,2.311464e-12,1.440358e-11,1.190980e-10,5.095945e-11,...,6.719200e-13,1.396617e-11,1.673780e-12,1.240151e-12,2.286295e-12,8.274464e-12,7.304889e-12,5.038531e-13,2.490866e-12,1.516462e-11
4,8000.0,8.053336e-11,5.396893e-11,3.157658e-10,6.331437e-11,2.552955e-10,2.218394e-12,1.364195e-11,1.168814e-10,4.722579e-11,...,6.965315e-13,1.448314e-11,1.594086e-12,1.212505e-12,2.094809e-12,8.266360e-12,7.597033e-12,4.356794e-13,2.749545e-12,1.559524e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,190000.0,6.268468e-11,2.407783e-10,1.287892e-10,2.739996e-10,4.451022e-11,3.473603e-12,1.243633e-11,1.210932e-10,6.662874e-12,...,7.322488e-13,7.473614e-12,1.582114e-12,1.266852e-12,2.314219e-12,7.752051e-12,7.825270e-12,5.002300e-13,1.152532e-11,1.236448e-11
96,192000.0,6.236680e-11,2.536299e-10,1.288454e-10,2.739372e-10,4.762429e-11,2.574099e-12,1.389727e-11,1.179844e-10,9.334006e-12,...,7.563474e-13,8.304173e-12,1.740047e-12,1.297001e-12,2.194628e-12,8.504321e-12,7.557819e-12,5.153411e-13,1.198802e-11,1.281320e-11
97,194000.0,6.034724e-11,2.291676e-10,1.295313e-10,2.821001e-10,4.821124e-11,2.041757e-12,1.423305e-11,1.151499e-10,8.009005e-12,...,7.641346e-13,8.133839e-12,1.755177e-12,1.237615e-12,1.845252e-12,7.326703e-12,8.969606e-12,5.036542e-13,1.262024e-11,1.297766e-11
98,196000.0,6.743471e-11,2.314253e-10,1.306056e-10,2.746544e-10,5.145656e-11,3.113668e-12,1.289336e-11,1.208145e-10,1.681773e-11,...,6.351157e-13,7.655775e-12,1.627590e-12,1.248765e-12,2.392381e-12,7.567776e-12,8.288189e-12,4.543345e-13,1.121582e-11,1.299821e-11
